Data Cleaning

In [63]:
import pandas as pd
import math
import ast

In [3]:
raw_datset = pd.read_csv("/Users/cameronfaulkner/Documents/Cogs 118a/Final Project/Scraped Tweets")

/Users/cameronfaulkner/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (22,23,25,28) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Select Relevant Features

keeping the following features: 

-rawContent

-replyCount

-reTweetCount

-likeCount

-quoteCount

-links

-media

-mentionedUsers

-hashtags 

-Party

In [4]:
raw_datset.columns

Index(['url', 'date', 'rawContent', 'renderedContent', 'id', 'user',
       'replyCount', 'retweetCount', 'likeCount', 'quoteCount',
       'conversationId', 'lang', 'source', 'sourceUrl', 'sourceLabel', 'links',
       'media', 'retweetedTweet', 'quotedTweet', 'inReplyToTweetId',
       'inReplyToUser', 'mentionedUsers', 'coordinates', 'place', 'hashtags',
       'cashtags', 'card', 'viewCount', 'vibe', 'State', 'Member of Congress',
       'Party'],
      dtype='object')

In [93]:
reduced_dataset = raw_datset[["rawContent","replyCount","retweetCount","likeCount","quoteCount","links","media","mentionedUsers","hashtags","Party"]]
#save off to csv
reduced_dataset.to_csv("Reduced Dataset",index=False)
reduced_dataset.head()

,rawContent,replyCount,retweetCount,likeCount,quoteCount,links,media,mentionedUsers,hashtags,Party
0,New Year’s Day in Seward overlooking the harbo...,44.0,12.0,239.0,2.0,NaN,[{'previewUrl': 'https://pbs.twimg.com/media/F...,NaN,NaN,R
1,Happy #NewYear!,19.0,4.0,22.0,1.0,NaN,NaN,NaN,['NewYear'],R
2,I championed provisions to strengthen Alaska’s...,9.0,5.0,19.0,0.0,NaN,NaN,NaN,NaN,R
3,"December: As senior Appropriator, I helped wri...",11.0,6.0,28.0,3.0,NaN,[{'previewUrl': 'https://pbs.twimg.com/media/F...,NaN,NaN,R
4,December: Congressman Don Young’s legacy will ...,5.0,9.0,46.0,1.0,NaN,[{'previewUrl': 'https://pbs.twimg.com/media/F...,NaN,NaN,R


In [94]:
#working off saved file
reduced_dataset = pd.read_csv("/Users/cameronfaulkner/Documents/Cogs 118a/Final Project/Reduced Dataset")

Transform Everything into Numerical Data:

In [95]:
#binarize party affiliation of tweet sender function
def party_binarize(party_list):
    output_list = []
    for party in party_list:
        if party == "R":
            output_list.append(1)
        else:
            output_list.append(0)
    return output_list

Binarizing the Party Data

In [96]:
binary_party = party_binarize(reduced_dataset["Party"])
reduced_dataset["Party"] = binary_party
reduced_dataset.head()

,rawContent,replyCount,retweetCount,likeCount,quoteCount,links,media,mentionedUsers,hashtags,Party
0,New Year’s Day in Seward overlooking the harbo...,44.0,12.0,239.0,2.0,NaN,[{'previewUrl': 'https://pbs.twimg.com/media/F...,NaN,NaN,1
1,Happy #NewYear!,19.0,4.0,22.0,1.0,NaN,NaN,NaN,['NewYear'],1
2,I championed provisions to strengthen Alaska’s...,9.0,5.0,19.0,0.0,NaN,NaN,NaN,NaN,1
3,"December: As senior Appropriator, I helped wri...",11.0,6.0,28.0,3.0,NaN,[{'previewUrl': 'https://pbs.twimg.com/media/F...,NaN,NaN,1
4,December: Congressman Don Young’s legacy will ...,5.0,9.0,46.0,1.0,NaN,[{'previewUrl': 'https://pbs.twimg.com/media/F...,NaN,NaN,1


Transforming Links, Media, and Hashtags into Numerical Values

In [97]:
#outputs number of items in a passed list, or 0 if the item isn't a list
def list_count(lst):
    if isinstance(lst,str):
        transform = ast.literal_eval(lst)
        return len(transform)
    return 0

In [98]:
#LINKS TRANSFORM, can only be done once

reduced_dataset["links"] = reduced_dataset["links"].apply(list_count)

#MEDIA TRANSFORM

reduced_dataset["media"] = reduced_dataset["media"].apply(list_count)

#HASHTAGS TRANSFORM

reduced_dataset["hashtags"] = reduced_dataset["hashtags"].apply(list_count)

#MENTIONEDUSERS TRANSFORMS
reduced_dataset["mentionedUsers"] = reduced_dataset["mentionedUsers"].apply(list_count)

In [99]:
reduced_dataset.head()

,rawContent,replyCount,retweetCount,likeCount,quoteCount,links,media,mentionedUsers,hashtags,Party
0,New Year’s Day in Seward overlooking the harbo...,44.0,12.0,239.0,2.0,0,3,0,0,1
1,Happy #NewYear!,19.0,4.0,22.0,1.0,0,0,0,1,1
2,I championed provisions to strengthen Alaska’s...,9.0,5.0,19.0,0.0,0,0,0,0,1
3,"December: As senior Appropriator, I helped wri...",11.0,6.0,28.0,3.0,0,1,0,0,1
4,December: Congressman Don Young’s legacy will ...,5.0,9.0,46.0,1.0,0,1,0,0,1


Check to see if each of our numerical features only contain floats

In [7]:
types_in_column = {"replyCount":[],"retweetCount":[],"likeCount":[],"quoteCount":[]}

In [8]:
for key in types_in_column.keys():
    types_in_column[key] = reduced_dataset[key].dtypes
types_in_column

{'replyCount': dtype('float64'),
 'retweetCount': dtype('float64'),
 'likeCount': dtype('float64'),
 'quoteCount': dtype('float64')}

As seen above, our numerical columns only include floats, as we wanted.

Next, we will check to see if the numerical values associated with every Tweet is in the correct range (greater than 0).

In [10]:
less_than_zero_in_column = {"replyCount":0,"retweetCount":0,"likeCount":0,"quoteCount":0}
for key in types_in_column.keys():
    count = reduced_dataset[key][reduced_dataset[key]<0].shape[0]
    less_than_zero_in_column[key] = count
less_than_zero_in_column

{'replyCount': 0, 'retweetCount': 0, 'likeCount': 0, 'quoteCount': 0}

Again, we see no values out of desired range in the scraped Tweets

In [21]:
#number of NaN Tweets: 0
print("number of NaN Tweets: " + str(reduced_dataset[reduced_dataset["rawContent"].notna() == False].shape[0]))
#number of Tweets with no content: 0
print("number of Tweets with no content: "+str(reduced_dataset["rawContent"][reduced_dataset["rawContent"].str.len()<1].shape[0]))

number of NaN Tweets: 0
number of Tweets with no content: 0


In [33]:
#median Tweet Count by user
print("Median Tweet Count by Member of Congress: "+ str(reduced_dataset.groupby("Member of Congress")["likeCount"].count().median()))

Median Tweet Count by Member of Congress: 1336.0


Save Changes

In [100]:
reduced_dataset.to_csv("Reduced Dataset",index=False)